In [7]:
import pandas as pd

In [8]:
import os
os.chdir('/data/sha0w/pyPath/jupyterProjects/tmp/PyCharmProject')

In [9]:
abalone_data = pd.read_csv("data/abalone.data", header=None, dtype=str)
lung_data = pd.read_csv("data/lung-cancer.data",header=None, dtype=str)
iris_data = pd.read_csv("data/iris.data",header=None, dtype=str)

In [10]:
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten


Using TensorFlow backend.


In [18]:
def test_layer(X_train, Y_train, label_num, flag = False) :
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=82)
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.5, random_state=82)
    print(X_train.shape)
    # print(Y_train[0])
    ret_dict =dict()
    for i in [3,4,5,6,7] : 
        model = Sequential()
        model.add(Dense(128, activation='relu', input_shape=X_train.shape[1:]))
        for j in range(i) : # add j label
            model.add(Dense(128, activation='relu'))
            model.add(Dropout(0.2))
        if flag : # regression
            model.add(Dense(1, activation='relu'))
            # model.summary()
            model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])
        else : # classification
            model.add(Dense(label_num, activation='softmax'))
            # model.summary()
            model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        hist = model.fit(X_train, Y_train, batch_size=32, epochs=30,
                         validation_data = (X_val,Y_val), callbacks=[],
                         verbose=0, shuffle=True)
        score = model.evaluate(X_test, Y_test, verbose=0)
        accuracy = 100 * score[1]
        print(accuracy)
        ret_dict[i] = (accuracy, hist)
    return ret_dict

def test_node(X_train, Y_train, label_num, flag = False) :
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=82)
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.5, random_state=82)
    print(X_train.shape)
    # print(Y_train[0])
    ret_dict =dict()
    for i in range(10) : 
        node_num = 2 ** ( i + 1)
        model = Sequential()
        model.add(Dense(node_num, activation='relu', input_shape=X_train.shape[1:]))
        model.add(Dropout(0.2))
        model.add(Dense(node_num, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(node_num, activation='relu'))
        model.add(Dropout(0.2))
        if flag : # regression
            model.add(Dense(1, activation='relu'))
            # model.summary()
            model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])
        else : # classification
            model.add(Dense(label_num, activation='softmax'))
            # model.summary()
            model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        hist = model.fit(X_train, Y_train, batch_size=32, epochs=30,
                         validation_data = (X_val,Y_val), callbacks=[],
                         verbose=0, shuffle=True)
        score = model.evaluate(X_test, Y_test, verbose=0)
        accuracy = 100 * score[1]
        print(accuracy)
        ret_dict[node_num] = (accuracy, hist)
    return ret_dict

In [19]:
import numpy as np
result_dict = dict()
for d in [iris_data, abalone_data, lung_data] :
    if d is iris_data :
        X = d.iloc[:,0:-1].values.astype('float')
        Y = d.iloc[:, -1].values.astype('int32')
        Y = to_categorical(Y, num_classes = 3)
        result_dict['iris_data'] = (test_node(X,Y,3),test_layer(X,Y,3))
    if d is abalone_data : # 回归问题
        X_ = d.iloc[:,0].values.astype('float')
        X_ = to_categorical(X_, num_classes = 3)
        X = d.iloc[:,1:-1,].values.astype('float')
        # print(X[0])
        X = np.concatenate((X_, X), axis=1)
        Y = d.iloc[ :,-1].values.astype('int32')
        result_dict['abalone_data'] = (test_node(X,Y,3, True),test_layer(X,Y,3, True))
    if d is lung_data :
        X = d.iloc[:,1:].values.astype('float')
        Y = d.iloc[:,0].values.astype('int32')
        Y = to_categorical(Y, num_classes = 3)
        result_dict['lung_data'] = (test_node(X,Y,3),test_layer(X,Y,3))

(67, 4)
32.35294222831726
36.764705181121826
29.411765933036804
64.70588445663452
64.70588445663452
97.0588207244873
67.64705777168274
91.17646813392639
86.76470518112183
76.47058963775635
(67, 4)
98.52941036224365
83.82353186607361
75.0
67.64705777168274
80.88235259056091
(1879, 10)
0.0
20.585106313228607
24.148936569690704
23.138298094272614
26.38297975063324
26.80850923061371
25.0
23.61702173948288
25.212764739990234
25.0
(1879, 10)
27.76595652103424
20.319148898124695
24.414893984794617
24.787233769893646
15.26595801115036
(14, 56)
14.28571492433548
14.28571492433548
21.42857164144516
28.57142984867096
28.57142984867096
28.57142984867096
35.71428656578064
28.57142984867096
28.57142984867096
28.57142984867096
(14, 56)
35.71428656578064
35.71428656578064
42.85714328289032
21.42857164144516
28.57142984867096


In [13]:
print(1)

1


In [119]:
import matplotlib.pyplot as plt
for d in ["iris_data", "abalone_data", "lung_data"] :
    s = "A"
    for data in result_dict[d]: 
        plt_x = []
        plt_y = []
        print(data)
        # plt.plot(x,y,"b--",linewidth=1)   #在当前绘图对象绘图（X轴，Y轴，蓝色虚线，线宽度）
        fig = plt.figure(figsize=(20,5)) #创建绘图对象
        for num,(acc, his) in data.items() :
            plt_x.append(acc / 100)
            print(num)
            plt_y.append(int(num))
            i = 1
            for wi in ["val_accuracy", "accuracy", "loss"] : 
                ax = fig.add_subplot(1, 3, i)
                ax.set_title(wi)
                ax.set_xlabel("epoch")
                ax.set_ylabel(wi)
                ax.plot(his.epoch,his.history[wi],'-',label=num,scaley=False)
                i += 1
                ax.legend()
        plt.show()
        fig.savefig(dpi=300,fname = d + s + "_1.jpg")
        fig_2 = plt.figure(figsize=(5,5))
        plt.plot(plt_y, plt_x,'-',scaley=False) #创建绘图对象
        plt.title(d + "在测试集上的准确度")
        plt.xlabel("数量")
        plt.ylabel("测试准确度")
        plt.show()
        fig_2.savefig(dpi=300,fname=d + s + "_2.jpg")
        s = "B"

{2: (32.35294222831726, <keras.callbacks.callbacks.History object at 0x7faa6770bef0>), 4: (36.764705181121826, <keras.callbacks.callbacks.History object at 0x7faa857e8160>), 8: (29.411765933036804, <keras.callbacks.callbacks.History object at 0x7faa7f9b26d8>), 16: (64.70588445663452, <keras.callbacks.callbacks.History object at 0x7faa8bb1ff60>), 32: (64.70588445663452, <keras.callbacks.callbacks.History object at 0x7faa65c31fd0>), 64: (97.0588207244873, <keras.callbacks.callbacks.History object at 0x7faa6454df28>), 128: (67.64705777168274, <keras.callbacks.callbacks.History object at 0x7faa62e98f60>), 256: (91.17646813392639, <keras.callbacks.callbacks.History object at 0x7faa617aaf98>), 512: (86.76470518112183, <keras.callbacks.callbacks.History object at 0x7faa60008e48>), 1024: (76.47058963775635, <keras.callbacks.callbacks.History object at 0x7faa5e891f28>)}
2


TypeError: cannot unpack non-iterable Figure object

<Figure size 1440x360 with 0 Axes>

<Figure size 360x360 with 0 Axes>

In [30]:
import sys
sys.path.append('C:\Python34\Lib\site-packages')
import matplotlib.pyplot as plt
from matplotlib.pylab import datestr2num

data = [1,2,3,4]
x = range(len(data))
x_date = [datestr2num(i) for i in data.index]
plt.style.use('ggplot')
plt.figure(figsize=(10,5))
plt.title("上证50指数历史最高价、收盘价走势折线图")
plt.xlabel("时间")
plt.xticks(rotation=45)
plt.ylabel("指数")
plt.plot_date(x_date,data['close'],'-',label="收盘价")
plt.plot_date(x_date,data['high'],'-',label="最高价")
plt.legend()
plt.grid(True)
    

TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
def plot_result(node_test) :
    accs = []
    for (num, (acc,his)) in node_test.items() : 
        accs.append(acc)
        pltdata = his["val_loss"]
        plt.plot_date(x_date,his["val_acc"],'-',label="val")
        plt.plot_date(x_date,his["loss"],'-',label="loss")
        plt.plot_date(x_date,his["acc"],'-',label="acc")
        plt.xlabel("时间")
        plt.xticks(rotation=45)
        plt.ylabel("指数")
        